In [29]:
import requests 
import pandas as pd
from requests import exceptions
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import argparse
import os
import sys
import psycopg2

In [2]:
EXCEPTIONS = set([IOError, FileNotFoundError, exceptions.RequestException, exceptions.HTTPError,
                 exceptions.ConnectionError, exceptions.Timeout])

In [511]:
class database:
    def __init__(self, dbname, user, password):
        host = "'localhost'"
        port="'5432'"
        db = "dbname =  '"+ str(dbname) + "' user= '"+str(user)+r"' password= '"+str(password) + "' host = "+str(host)+" port = " + str(port)
        db = str(db)
        print (db)
        #conn = psycopg2.connect("dbname = 'web_scrapper_db' user='postgres' password='postgresql123' host = 'localhost' port='5432'")
        conn = psycopg2.connect(db)
        cur = conn.cursor()
        cur.execute("CREATE TABLE IF NOT EXISTS real_madrid (Name_of_video text, search_criteria text, Date_added text, Duration text, cast_name text, category text, description text, Tags text, Fembed_Link text)")
        conn.commit()
        conn.close()

    def insert_old_copy(Name_of_video,Date_added, Duration,cast_name,Category, Description, Tags, search_criteria, Fembed_Link):
        conn = psycopg2.connect("dbname = 'web_scrapper_db' user='postgres' password='postgres123' host = 'localhost'port='5432'")
        cur = conn.cursor() 
        cur.execute("INSERT INTO real_madrid VALUES (%s , %s, %s, %s, %s, %s, %s, %s, %s)",(Name_of_video,Date_added, Duration,cast_name,Category, Description, Tags, search_criteria, Fembed_Link))
        conn.commit()
        conn.close()

    def view():
        conn = psycopg2.connect("dbname = 'web_scrapper_db' user='postgres' password='postgres123' host = 'localhost'port='5432'")
        cur = conn.cursor()
        cur.execute("SELECT * FROM real_madrid")
        view = cur.fetchall()
        conn.close()
        return view
    
    
    def insert(Name_of_video,Date_added, Duration,cast_name,Category, Description, Tags, search_criteria, Fembed_Link):
        conn = psycopg2.connect("dbname = 'web_scrapper_db' user='postgres' password='postgres123' host = 'localhost'port='5432'")
        cur = conn.cursor() 
        cur.execute("SELECT * FROM real_madrid WHERE Name_of_video LIKE %s", [Name_of_video])
        exist = cur.fetchall()
        if len (exist) > 1:
            pass
        else:
            cur.execute("INSERT INTO real_madrid VALUES (%s , %s, %s, %s, %s, %s, %s, %s, %s)",(Name_of_video,Date_added, Duration,cast_name,Category, Description, Tags, search_criteria, Fembed_Link))
            conn.commit()
        conn.close()
        
    
database('web_scrapper_db' , 'postgres', 'postgres123') #pass your credentials in strings like this

dbname =  'web_scrapper_db' user= 'postgres' password= 'postgres123' host = 'localhost' port = '5432'


In [4]:
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_filename

===============================================JAVqd.COM==============================================================

In [11]:
search_criteria = "beautiful-girl" #put '-' instead of space
url = "https://www.javqd.com/search/movie/"+ search_criteria +"/page-1.html"
print (url)
web= requests.get(url)
content = web.content
full_search_return = soup(content, "html.parser")
video_content_list = full_search_return.find_all("div", {"class":"col-lg-3 col-md-4 col-sm-6 videoitem"})
total_num_dis_page = 32
total_number_result_page = full_search_return.find("div", {"class": "col-xs-12 border"}).text.split(' ')[1]
total_number_result_page = int (int(total_number_result_page)/total_num_dis_page)
total_number_result_page
if total_number_result_page ==  0:
    total_number_result_page = 1
print(total_number_result_page)

https://www.javqd.com/search/movie/beautigul-girl/page-1.html
82


In [12]:
for page in range (total_number_result_page):
    if page > 12:
        url = "https://www.javqd.com/search/movie/"+ search_criteria +"/page-" + str(page)+".html"
        web= requests.get(url)
        content = web.content
        full_search_return = soup(content, "html.parser")
        video_content_list = full_search_return.find_all("div", {"class":"col-lg-3 col-md-4 col-sm-6 videoitem"})
        total_num_dis_page = 32
        total_number_result_page = full_search_return.find("div", {"class": "col-xs-12 border"}).text.split(' ')[1]
        total_number_result_page = int (int(total_number_result_page)/total_num_dis_page)
        total_number_result_page
        try:
            for video in range (total_num_dis_page):
                link_video_main_page = str(video_content_list[video].find("a")).split('"')[1]
                link_video_main_page = "https://www.javqd.com" + link_video_main_page
                #print (link_video_main_page)
                web_main_page = requests.get(link_video_main_page)
                content_main_page = web_main_page.content
                content_parsed = soup(content_main_page, "html.parser")
                link = str(content_parsed.find("textarea" ,{"class":"select-all"})).split('"')[13]
                #print (link)
                browser = webdriver.Chrome(r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
                wait = WebDriverWait(browser, 10)
                browser.get(link)
                elements = browser.find_element_by_css_selector("#external-embed").get_attribute("src")
                fembed_link = elements
                print (fembed_link)
                browser.close()
                Name_of_video = content_parsed.find("h2", {"class":"info-box-heading"}).text
                #print(Name_of_video)
                v_info = content_parsed.find("div", {"class": "tab-content tab-description active"}).text.lower().split("\n")

                try:
                    for item in v_info:
                        if item == "added:" or item == "added":
                            index = v_info.index(item)
                            date_added = v_info[int(index) + 1]
                            #print (date_added)
                except Exception as e:
                        date_added = "NaN"

                try:
                    for item in v_info:
                        if item == "duration:" or item == "duration":
                            index = v_info.index(item)
                            duration = v_info[int(index) + 1]
                            #print (duration)
                except Exception as e:
                        duration = "NaN"

                try:
                    for item in v_info:
                        if item == "cast:" or item == "cast":
                            index = v_info.index(item)
                            cast_name = v_info[int(index) + 1]
                            #print (cast_name)
                except Exception as e:
                        cast_name = "NaN"

                try:
                    for item in v_info:
                        if item == "category:" or item == "category":
                            index = v_info.index(item)
                            category = v_info[int(index) + 1]
                            #print (category)
                except Exception as e:
                        category = "NaN"

                try:
                    for item in v_info:
                        if item == "description:" or item == "description":
                            index = v_info.index(item)
                            description = str(v_info[int(index) + 1])
                            #print (description)
                except Exception as e:
                        description = "NaN"

                try:
                    for item in v_info:
                        if item == "tags:" or item == "tags":
                            index = v_info.index(item)
                            tags = v_info[int(index) + 1: len(v_info)]
                            #print (tags)
                except Exception as e:
                        Tags = "NaN"


                database.insert(Name_of_video , search_criteria , date_added , duration , cast_name , category , description , tags , fembed_link)
        except Exception as e:
            continue
    else:
        page = page + 1
        print(page)

1
2
3
4
5
6
7
8
9
10
11
12
13
https://www.fembed.com/v/8gvz1r34y9d
https://www.fembed.com/v/1l96kdyzlo5
https://www.fembed.com/v/zyvn4lzq891
https://www.fembed.com/v/05olelkknv6
https://www.fembed.com/v/05olex5qnv6
https://www.fembed.com/v/1xoqdlqlxv4


KeyboardInterrupt: 

In [36]:
data = pd.DataFrame(database.view(), columns = ["Name_of_video","Date_added", "Duration","cast_name","Category", "Description", "Tags", "search_criteria", "Fembed_Link"])

In [39]:
data["Name_of_video"][0].replace(" ", "-")

'FHD-SkyuShiroto-SUPA-340-I-Want-You-To-Be-Inserted-Cum-Inside-SEX-Best-4-Hour-With-Horny-Amateur-Girls-Admitting-Insert-Yourself'

In [105]:
#code to get thumbnail image on javqd.com
for video in range(0, 5) :   
    url = "https://www.javqd.com/movie/"+ data["Name_of_video"][video].replace(" ", "-").replace(",", "").replace(".", "-")
    #print (url)
    web= requests.get(url)
    content = web.content
    res = soup(content, "html.parser")
    res = res.find_all("script", {"type": "application/ld+json"})[2].text
    res = res.split('"image": ')[-1].rstrip("\n")
    print(res)
    

"https://jqd.cdn-qdnetwork.com/img/movie/20180707/fhd-skyushiroto-supa-340-i-want-you-to-be-inserted-cum-inside-sex-best-4-hour-with-horny-amateur-girls-admitting-insert-yourself_1530929826.png"
            }
        }
    ]
}
"https://jqd.cdn-qdnetwork.com/img/movie/20180629/fhd-s-cute-sqte-215-horny-desire-of-a-pheromone-girl-who-puts-a-fire-on-a-cockroach_1530235924.png"
            }
        }
    ]
}
"https://jqd.cdn-qdnetwork.com/img/movie/20180618/fhd-s-cute-sqte-213-that-beautiful-girl-sensitive-horny-ecology-of-a-girl-who-does-not-stop-pleasure_1529289790.png"
            }
        }
    ]
}
"https://jqd.cdn-qdnetwork.com/img/movie/20180522/fhd-s-cute-sqte-210-i-made-a-gonzo-picture-of-my-favorite-canojo-beautiful-girls-horny-looks-like-a-close-friend-icha-cha-play_1526976810.png"
            }
        }
    ]
}
"https://jqd.cdn-qdnetwork.com/img/movie/20180518/heydouga-4037-ppv381-nanao-gachin-girls-nana-horny-everyday-123_1526652720.png"
            }
        }
    ]
}


=========================================Final Link to download video=================================================

###final link needed for downloading
browser = webdriver.Chrome(r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
wait = WebDriverWait(browser, 10)
browser.get(elements)
elements = browser.find_element_by_css_selector("video").get_attribute("src")
print (elements)
browser.close()

download_file(elements)

===========================================West(yourporn.sexy)=================================================

#get full website
web = requests.get("https://yourporn.sexy/sexy-CHINESE-GIRL.html")
content = web.content
full_content = soup(content, "html.parser")
content_1 = full_content.find_all("div", {"class" : "post_el_small"})

#get prelink
view_1 = content_1[0].find_all("a")
prelink = ""
count= 0
for links in view_1:
    if count == 0:
        string = str(links)
        if "post" in string:
            prelink = string
            #print(prelink)
            count +=1

#https://s12-1.trafficdeposit.com/bvideo/OveI6-OWRtcnl2JyZx29Fg/1531539661/5a0b5ff2d94da/5a84e8b3790a5.mp4

#get link for video
prelink = prelink.split()
link = prelink[1]
link = link[6:len(link) - 6]

link = "https://yourporn.sexy"+link
link

web_2 = requests.get("https://yourporn.sexy/post/5a84e8b3790a5.html?sk=sexy-CHINESE-GIRL&amp;so=0&amp;ss=latest")
content_2 = web_2.content
full_content_1 = soup(content_2, "html.parser")

#get all the sub_directiorys

"https://s12-1.trafficdeposit.com/bvideo/uCPg0lLvdc4mRUHDttbb-Q/1531585511/5a0b5ff2d94da/5a84e8b3790a5.mp4"
"https://con1.trafficdeposit.com/bvideo/con2/con3/con4/con5.mp4"

token = full_content_1.find_all("span",  {"class":"vidsnfo"})
con1 = str(token).split("/")[2][:-1]
con2 = str(token).split("/")[3][:-1]
con3 = str(token).split("/")[4][:-1]
con5 = str(token).split('"')[3]
token_1 = full_content_1.find_all("div", {"class":"pes_author_div pes_edit_div transition"})
con4 = str(token_1).split('"')[3]
print(con1, con2, con3, con4, con5)
final_link = "https://"+con1+".trafficdeposit.com/bvideo/"+con2+"/"+con3+"/"+con4+"/"+con5+".mp4"
print (final_link)

download_file(final_link)

===========================================================================================================

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import DesiredCapabilities
import json
from collections import OrderedDict
import re
import ast

In [ ]:
#python performance review code by jacob and tom

chrome_options = Options()
chrome_options.add_argument("--headless")
caps = DesiredCapabilities.CHROME
caps["loggingPrefs"] = {"performance": "ALL"}
browser = webdriver.Chrome(chrome_options=chrome_options, desired_capabilities= caps, executable_path = r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
wait = WebDriverWait(browser, 10)
browser.get("https://www5.javfinder.is/category/teacher")
for i in browser.get_log("performance"):
    print(i)

In [ ]:
#code to get fembedlink

fembed_link = []
caps = DesiredCapabilities.CHROME
caps["loggingPrefs"] = {"performance": "ALL"}
browser = webdriver.Chrome(chrome_options=chrome_options, desired_capabilities= caps, executable_path = r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
wait = WebDriverWait(browser, 10)
browser.get("https://yourporn.sexy/post/5a84e8b3790a5.html?sk=sexy-CHINESE-GIRL&amp;so=0&amp;ss=latest")
for i in browser.get_log("performance"):
    for j in json.loads(i["message"]).values():
        if  type(j) == dict :
            for z in j["params"].values():
                if  type(z) == dict :
                    for y in z:
                        if y == "url":
                            #if "embed.com" in z[y]:
                            fembed_link.append(z[y])
#fembed_link = fembed_link[0]  
fembed_link

In [336]:
browser = webdriver.Chrome(chrome_options=chrome_options, executable_path = r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
wait = WebDriverWait(browser, 10)
browser.get("https://www5.javfinder.is/movie/watch/fhd-hodv-21315-my-sister-is-friend-who-shows-innocently-while-trying-on-a-swimwear-in-our-living-room-i-got-an-erection-witnessing-the-swimsuit-butt.html")
elements = browser.find_element_by_css_selector(".col-md-5").find_element_by_xpath("img").get_attribute("src")

In [337]:
elements

'https://images1-focus-opensocial.googleusercontent.com/gadgets/proxy?container=focus&url=https://cdnfd.me/img/movie/20180808/fhd-hodv-21315-my-sister-is-friend-who-shows-innocently-while-trying-on-a-swimwear-in-our-living-room-i-got-an-erection-witnessing-the-swimsuit-butt_1533695752.png'

============================================New Code for Javfinder ==================================================

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import DesiredCapabilities
import json
from collections import OrderedDict
import re
import ast
import requests 
import pandas as pd
from requests import exceptions
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import argparse
import os
import sys
import psycopg2
import MySQLdb

/Applications/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [34]:
search_criteria = "beautiful-girl" #put '-' instead of space
url = "https://www.javfinder.is/search/movie/"+ search_criteria +"/page-1.html"
print (url)
web= requests.get(url)
content = web.content
full_search_return = soup(content, "html.parser")
video_content_list = full_search_return.find_all("div", {"class":"col-md-3 text-center main-item"})
total_num_dis_page = 32
total_number_result_page = full_search_return.find("div", {"class": "col-xs-12 border"}).text.split(' ')[1]
total_number_result_page = int (int(total_number_result_page)/total_num_dis_page)
total_number_result_page
if total_number_result_page ==  0:
    total_number_result_page = 1
print(total_number_result_page)

https://www.javfinder.is/search/movie/beautiful-girl/page-1.html
19


In [163]:
for page in range (total_number_result_page):
    url = "https://www.javfinder.is/search/movie/"+ search_criteria +"/page-" + str(page)+".html"
    web= requests.get(url)
    content = web.content
    full_search_return = soup(content, "html.parser")
    video_content_list = full_search_return.find_all("div", {"class":"col-md-3 text-center main-item"})
    total_num_dis_page = 32
    total_number_result_page = full_search_return.find("div", {"class": "col-xs-12 border"}).text.split(' ')[1]
    total_number_result_page = int (int(total_number_result_page)/total_num_dis_page)
    total_number_result_page
    try:
        for video in range (total_num_dis_page):
            link_video_main_page = str(video_content_list[video].find("a")).split('"')[3]
            link_video_main_page = "https://www.javfinder.is" + link_video_main_page
            #print (link_video_main_page)
            web_main_page = requests.get(link_video_main_page)
            content_main_page = web_main_page.content
            content_parsed = soup(content_main_page, "html.parser")
            link = str(content_parsed.find("textarea" ,{"class":"select-all"})).split('"')[13]
            #print (link)
            fembed_link_x = []
            caps = DesiredCapabilities.CHROME
            caps["loggingPrefs"] = {"performance": "ALL"}
            browser = webdriver.Chrome(chrome_options=chrome_options, desired_capabilities= caps, executable_path = r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
            wait = WebDriverWait(browser, 5)
            browser.get(link_video_main_page)
            thumb_img = browser.find_element_by_css_selector(".col-md-5").find_element_by_xpath("img").get_attribute("src")
            browser.close() 
            print(thumb_img) #str(content_parsed.find("div", {"class": "wrap-meta"}).find("img")).split("url=")[1]
            browser = webdriver.Chrome(chrome_options=chrome_options, desired_capabilities= caps, executable_path = r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
            wait = WebDriverWait(browser, 5)
            browser.get(link)
            for i in browser.get_log("performance"):
                for j in json.loads(i["message"]).values():
                    if  type(j) == dict :
                        for z in j["params"].values():
                            if  type(z) == dict :
                                for y in z:
                                    if y == "url":
                                        if "embed.com" in z[y]:
                                            fembed_link_x.append(z[y])
            fembed_link = fembed_link_x[0]  
            print (fembed_link)
            browser.close()
            Name_of_video = content_parsed.find("h1").text
            print(Name_of_video)
            v_info = content_parsed.find("div", {"class": "wrap-meta"}).text.split("\n")
            try:
                for item in v_info:
                    if item == "Release" or item == "Released":
                        index = v_info.index(item)
                        date_added = v_info[int(index) + 2]
                        print (date_added)
            except Exception as e:
                    date_added = "NaN"
            try:
                for item in v_info:
                    if item == "meta:" or item == "meta":
                        index = v_info.index(item)
                        duration = v_info[int(index) + 2].strip().split(" ")[1]
                        print (duration)
            except Exception as e:
                duration = "NaN"

            try:
                for item in v_info:
                    if item == "Actor:" or item == "Actor":
                        index = v_info.index(item)
                        cast_name = v_info[int(index) + 2]
                        print (cast_name)
            except Exception as e:
                cast_name = "NaN"
            try:
                for item in v_info:
                    if item == "Genre:" or item == "Genre":
                        index = v_info.index(item)
                        category = v_info[int(index) + 2]
                        print (category)
            except Exception as e:
                category = "NaN"
            try:
                for item in v_info:
                    if item == "Studio:" or item == "Studio":
                        index = v_info.index(item)
                        studio = v_info[int(index) + 2]
                        print (studio)
            except Exception as e:
                studio = "NaN"
            try:
                description = content_parsed.find("div", {"class": "main-content"}).text.split("\n")
                print (description)
            except Exception as e:
                description = "NaN"

            try:
                tags = content_parsed.find("div", {"class": "wrap-tags clearfix"}).text.split("\n")
                print (tags)
            except Exception as e:
                    Tags = "NaN"
            print ("===================================")
    except Exception as e:
        print("=================================")
        continue
    

https://images1-focus-opensocial.googleusercontent.com/gadgets/proxy?container=focus&url=https://cdnfd.me/img/movie/20180810/momotaroeizo-ymdd-135-miraculous-beautiful-girl-run-min-mini-av-debut-from-the-taiwan-where-the-local-girl-excavated-and-the-team-will-go_1533886435.png
https://qdembed.com/v/4lv0gk2kxoq
MomotaroEizo YMDD-135 Miraculous Beautiful Girl, Run Min Mini AV Debut From The Taiwan Where The Local Girl Excavated And The Team Will Go
                                                    Aug 10th, 2018                                                
Lan Minmei 
Asian, Beautiful Girl, Big tits, Blowjob, Censored, Cowgirl, Creampie, Cumshot, Cunnilingus, Doggy Style, Fingering, Handjob, Japanese 
Momotaro Eizo 
['', 'Watch Jav Online 【Cuteness Explosion! Taiwanese daughter】 There is no man in Japan. I have debuted Min Mai AV! Excavate a young girl from town in Yokohama Chinatown. After inviting Mr. Min Mai (a 21-year-old from Taiwan) who is part-time at Chinatown to date and un

KeyboardInterrupt: 

In [161]:
Name_of_video , search_criteria , date_added , cast_name , category , description , tags , fembed_link


('FC2 PPV 894185 Insert into the anal pure amateur girl anal! Finally cum shot in a beautiful shaved punk',
 'beautiful-girl',
 '                                                    Aug 8th, 2018                                                ',
 'Porn Star ',
 '69 Style, Amateur, Beautiful Girl, Big Natural Tits, Big tits, Blowjob, College, Cowgirl, Creampie, Doggy Style, Handjob, Uncensored ',
 ['',
  'Watch Jav Online FC2 PPV 894185 Insert into the anal pure amateur girl anal! Finally cum shot in a beautiful shaved punk\xa0FC2-PPV-894185 清純系素人娘のアナルに挿入！最後は綺麗なパイパンまんこに中出し by Javfinder',
  ''],
 ['',
  'Fc2 Ppv 894185',
  'Insert',
  'Into',
  'Anal Pure',
  'Amateur Girl',
  'Anal',
  'Finally',
  'Cum Shot',
  'Beautiful',
  'Shaved Punk',
  ''],
 'https://qdembed.com/v/05olqqdenv6')

In [162]:
class database:
    def __init__(self, dbname, user, password):
        host = "'localhost'"
        port="'5432'"
        db = "dbname =  '"+ str(dbname) + "' user= '"+str(user)+r"' password= '"+str(password) + "' host = "+str(host)+" port = " + str(port)
        db = str(db)
        print (db)
        #conn = psycopg2.connect("dbname = 'web_scrapper_db' user='postgres' password='postgresql123' host = 'localhost' port='5432'")
        conn = psycopg2.connect(db)
        cur = conn.cursor()
        cur.execute("CREATE TABLE IF NOT EXISTS wp_posts (ID integer,post_author text,post_date text,post_date_gmt text, post_content text, post_title text, post_excerpt text, post_status text, comment_status text, ping_status text, post_password text, post_name text, to_ping text, pinged text, post_modified text, post_modified_gmt text, post_content_filtered text, post_parent text, guid text, menu_order text, post_type text, post_mime_type text, comment_count text)")
        cur.execute("CREATE TABLE IF NOT EXISTS wp_postsmeta (post_id text, meta_key text, meta_value text)")
        conn.commit()
        conn.close()
database('web_scrapper_db' , 'postgres', 'postgres123') #pass your credentials in strings like this

dbname =  'web_scrapper_db' user= 'postgres' password= 'postgres123' host = 'localhost' port = '5432'


In [513]:
post_content = """<iframe src= {} width="848" height="477" frameborder="0" allowfullscreen="allowfullscreen"><span data-mce-type="bookmark" style="display: inline-block; width: 0px; overflow: hidden; line-height: 0;" class="mce_SELRES_start">﻿</span></iframe>""".format(fembed_link) 

'<iframe src= y width="848" height="477" frameborder="0" allowfullscreen="allowfullscreen"><span data-mce-type="bookmark" style="display: inline-block; width: 0px; overflow: hidden; line-height: 0;" class="mce_SELRES_start">\ufeff</span></iframe>'

In [ ]:
"ID text,post_author text,post_date text,post_date_gmt text, post_content text, post_title text, post_excerpt text, post_status text, comment_status text, ping_status text, post_password text, post_name text, to_ping text, pinged text, post_modified text, post_modified_gmt text, post_content_filtered text, post_parent text, guid text, menu_order text post_type text, post_mime_type text, comment_count text

In [515]:
post_id text, meta_key text, meta_value text

TypeError: __init__() missing 3 required positional arguments: 'dbname', 'user', and 'password'

In [1]:
import MySQLdb

In [241]:
class sqldb:
    def __init__ (self):
        conn = MySQLdb.connect(host="192.168.2.12",port = 3306, user="realmadrid", passwd="jacobjoetom", db="realmadrid",
                              use_unicode= True, charset = "utf8")        # name of the data base
        cur = conn.cursor()
        conn.close()

    def insert_old_copy(Name_of_video,Date_added, Duration,cast_name,Category, Description, Tags, search_criteria, Fembed_Link):
        conn = MySQLdb.connect(host="192.168.2.12",port = 3306, user="realmadrid", passwd="jacobjoetom", db="realmadrid",
                              use_unicode= True, charset = "utf8")        # name of the data base
        cur = conn.cursor()
        cur.execute("INSERT INTO real_madrid VALUES (%s , %s, %s, %s, %s, %s, %s, %s, %s)",(Name_of_video,Date_added, Duration,cast_name,Category, Description, Tags, search_criteria, Fembed_Link))
        conn.commit()
        conn.close()

    def view():
        conn = MySQLdb.connect(host="192.168.2.12",port = 3306, user="realmadrid", passwd="jacobjoetom", db="realmadrid",
                              use_unicode= True, charset = "utf8")        # name of the data base
        cur = conn.cursor()
        cur.execute("SELECT * FROM wp_posts")
        view = cur.fetchall()
        conn.close()
        return view
    
    
    def insert_wppost(Name_of_video, date_added, content, postname):
        conn = MySQLdb.connect(host="192.168.2.12",port = 3306, user="realmadrid", passwd="jacobjoetom", db="realmadrid",
                              use_unicode= True, charset = "utf8")        # name of the data base
        cur = conn.cursor()
        cur.execute("SELECT * FROM `wp_posts` WHERE `post_title` = (%s)", [Name_of_video])
        exist = cur.fetchall()
        if len (exist) > 1:
            pass
        else:
            cur.execute("INSERT INTO `wp_posts`(`ID`, `post_title`, `post_date`, `post_content`, `post_name`) VALUES (%s, %s, %s, %s, %s)",[200, Name_of_video, date_added, content, postname])
            conn.commit()
        conn.close()

    def insert_wppostmeta(thumbimg):
        conn = MySQLdb.connect(host="192.168.2.12",port = 3306, user="realmadrid", passwd="jacobjoetom", db="realmadrid",
                              use_unicode= True, charset = "utf8")        # name of the data base
        cur = conn.cursor()
        cur.execute("SELECT * FROM `wp_postmeta` WHERE `post_id` = (%s) and `meta_key` = (%s)", [200, "fifu_image_url"])
        exist = cur.fetchall()
        if len (exist) > 1:
            pass
        else:
            cur.execute("INSERT INTO `wp_postmeta`wp_terms, `meta_value`, `meta_key`) VALUES (%s, %s, %s)",[200, thumbimg, "fifu_image_url"])
            conn.commit()
        conn.close()
    
    def insert_wpterm(cast_name):
        term_id_list = []
        conn = MySQLdb.connect(host="192.168.2.12",port = 3306, user="realmadrid", passwd="jacobjoetom", db="realmadrid",
                              use_unicode= True, charset = "utf8")        # name of the data base
        cur = conn.cursor()
        for num, actor in enumerate(cast_name.split(",")):
            j = str(actor).strip()
            i = j.replace(" ", "-").lower()
            cur.execute("SELECT `term_id` FROM `wp_terms` WHERE `slug` = (%s)", [i])
            exist = cur.fetchall()
            if len(exist) > 0:
                term_id_list.append([x[0] for x in exist])
            else:
                cur.execute("INSERT INTO `wp_terms` (`term_id`, `name`, `slug`) Values (%s, %s, %s)", [300 + num, j, i])
                #cur.execute("INSERT INTO `wp_term`(`term_id`, `name`, `slug`) VALUES (%s, %s, %s)",[200, thumbimg, "fifu_image_url"])
                conn.commit()
                term_id_list.append(300 + num)
            
        for num,   
                
                
                
        return term_id_list

In [143]:
from datetime import datetime

content = """<iframe src= {}{}{} width="848" height="477" frameborder="0" allowfullscreen="allowfullscreen"><span data-mce-type="bookmark" style="display: inline-block; width: 0px; overflow: hidden; line-height: 0;" class="mce_SELRES_start">﻿</span></iframe>""".format('"', fembed_link, '"')
content
postname = str(fembed_link).split("/")[-1]
postname
dateadded = datetime.strftime(datetime.strptime(str(date_added).strip().replace(",", ""), '%b %dth %Y'), "%Y-%m-%d")
dateadded

'2018-08-10'

In [128]:
sqldb.insert_wppost(Name_of_video, dateadded, content, postname)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: Warning: (1364, "Field 'post_excerpt' doesn't have a default value")
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: Warning: (1364, "Field 'to_ping' doesn't have a default value")
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: Warning: (1364, "Field 'pinged' doesn't have a default value")
/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: Warning: (1364, "Field 'post_content_filtered' doesn't have a default value")


In [156]:
sqldb.insert_wppostmeta(thumb_img)

In [242]:
cast = "ass, joe"
sqldb.insert_wpterm(cast)

[[12, 300], [210, 301]]